In [1]:
import typing

import pandas as pd

import config
import src
import requests
import tqdm
import json
import numpy as np
import logging

#import cltrier_lib as lib

import yaml
pd.set_option('display.max_colwidth', 100) 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\sstolwi\Git

In [2]:
CFG = config.Config()

In [3]:
#load data
reddit3000: pd.DataFrame = pd.read_parquet(f'{CFG.report_dir}/reddit3000.parquet')
reddit3000.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI
0,t3_1bx6qw,,100movies365days,DTX120: #87 - Nashville,none,announcement,True,NaN,None,[],"[0.3085225224494934, 0.0025453902781009674, -0..."
1,t1_c9b2nyd,I've wanted to watch this for a long time. I w...,100movies365days,DTX120: #87 - Nashville,t3_1bx6qw,elaboration,None,1.0,mcgrewf10,"[""The user has wanted to watch something for a...","[0.4254729151725769, -0.01032189279794693, -0...."
2,t1_c9b30i1,You strike me as the type who would appreciate...,100movies365days,DTX120: #87 - Nashville,t1_c9b2nyd,elaboration,None,2.0,None,"[""Magnolia has a similar structure to Altman's...","[0.7148110270500183, -0.15317122638225555, -0...."
3,t1_c9b6sj0,"Yeah, I've always heard that Altman was famous...",100movies365days,DTX120: #87 - Nashville,t1_c9b30i1,elaboration,None,3.0,mcgrewf10,"[""Altman is famous for his ensemble casts"", ""T...","[0.7345885634422302, -0.177871972322464, -0.60..."
4,t3_omv7p,"Alright guys, little background about myself. ...",100sets,"Male, 23 years old. Going for 100 sets!",none,announcement,True,NaN,Keatonus,"[""The writer has had modest success with women...","[0.5479655265808105, -0.197927325963974, -0.42..."


In [4]:
#select the rows in the dataset that have the a showName with more than cutoff comments:   
cutoff = 40
selectedposts = reddit3000[reddit3000['thread_title'].isin(reddit3000['thread_title'].value_counts()[reddit3000['thread_title'].value_counts() >= cutoff].index)]

In [5]:
selectedposts.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI
1175,t3_17feeu,I'm rooted on an S3 and debating what ROM I'd ...,android,What ROM are you currently running and what do...,none,question,True,NaN,RipperDaVe,[],"[0.3085225224494934, 0.0025453902781009674, -0..."
1176,t1_c84zkny,ParanoidKANGdroid. Has AOKP paranoid android a...,android,What ROM are you currently running and what do...,t3_17feeu,answer,None,1.0,None,"[""AOKP paranoid android and CM10 can be combin...","[0.6152559518814087, -0.22806480526924133, -0...."
1177,t1_c84znti,Isn't this kind of like PAC (Paranoid/AOKP/CM)?,android,What ROM are you currently running and what do...,t1_c84zkny,question,None,2.0,OmegaVesko,"[""This is similar to PAC (Paranoid/AOKP/CM).""]","[0.4787346422672272, -0.1805042177438736, -0.4..."
1178,t1_c850f55,"Omg. That's what it should be called, way cool...",android,What ROM are you currently running and what do...,t1_c84znti,appreciation,None,3.0,None,"[""ParanoidKANGdroid is not as cool as the sugg...","[0.35291609168052673, -0.2874097526073456, -0...."
1179,t1_c850if1,http://forum.xda-developers.com/showthread.php...,android,What ROM are you currently running and what do...,t1_c850f55,elaboration,None,4.0,OmegaVesko,[],"[0.3085225224494934, 0.0025453902781009674, -0..."


In [6]:
selectedposts['thread_title'].value_counts()

thread_title
What ROM are you currently running and what do you like/dislike about it?          41
Get rid of unwanted system apps (adb shell pm hide)                                41
If your country has a 30-35 hour work week, what do you do with your free time?    41
Name: count, dtype: int64

In [7]:
#try new Achim approach of first expanding the meaning of the post based on context and then classifying the addition of new information per post:


In [8]:
SYSTEM_expansion: str = \
    """
        Instruction:\nYou verbalize the meaning of a social media post given the preceding comments in the thread. You will be provided with a thread of comments enclosed in chevrons  <..> and a target post enclosed in double chevrons <<...>>.\nYou respond with the target reply-post, but, if it contains words alluding to the original post, you expand on them so that their meaning is preserved without the preceding comment. Your response is worded as a social media post. If the meaning of the target reply-post is not affected by the context of the original post, you reply with a verbatim copy of the target reply-post.\\n\\nText:"
	"""

In [9]:
MODEL: str = 'llama3.1:8b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'


In [10]:
options_low = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low)

In [20]:
requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODEL,
                                'system': SYSTEM_expansion,
                                'prompt': f'"Thread":\n<{selectedposts["body"][:5].to_list()}>, "Target reply":<<{selectedposts["body"][5:6].to_list()}>>',
                                'options': options_low
                                }).json()

{'id': '18fd5568-914f-11ef-b791-b5b8664f6fd3',
 'timestamp': '2024-10-23T14:57:17.947611',
 'model': 'llama3.1:8b-instruct-q6_K',
 'prompt': [{'role': 'system',
   'content': '\n        Instruction:\nYou verbalize the meaning of a social media post given the preceding comments in the thread. You will be provided with a thread of comments enclosed in chevrons  <..> and a target post enclosed in double chevrons <<...>>.\nYou respond with the target reply-post, but, if it contains words alluding to the original post, you expand on them so that their meaning is preserved without the preceding comment. Your response is worded as a social media post. If the meaning of the target reply-post is not affected by the context of the original post, you reply with a verbatim copy of the target reply-post.\\n\\nText:"\n\t'},
  {'role': 'user',
   'content': '"Thread":\n<["I\'m rooted on an S3 and debating what ROM I\'d like to flash, any and all opinions would be appreciated, thanks!\\n\\nEdit: Happy

In [22]:
selectedposts["body"][:5].to_list()

["I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appreciated, thanks!\n\nEdit: Happy to hear ideas from other phones as well, thanks for the input",
 'ParanoidKANGdroid. Has AOKP paranoid android and CM10 combined. Absolutely genius. And it is stable! The customization is incredible. On jellybean too.\n\n**EDIT:** link for OP, on the I747 GS3 \n\nhttp://forum.xda-developers.com/showthread.php?t=1944523',
 "Isn't this kind of like PAC (Paranoid/AOKP/CM)?",
 "Omg. That's what it should be called, way cooler than paranoidKANGdroid.",
 'http://forum.xda-developers.com/showthread.php?t=2015607']

In [21]:
selectedposts["body"][5:6].to_list()

['Thanks this looks really good']

In [26]:
selectedposts['preceding_index'] = selectedposts.index.to_series().shift(1)

AttributeError: 'Series' object has no attribute 'as_type'

In [37]:
selectedposts['preceding_index'].isna().sum()

np.int64(1)

In [40]:
selectedposts.drop(columns=['preceding_index'], inplace=True)

C:\Users\sstolwi\AppData\Local\Temp\ipykernel_15560\714106042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selectedposts.drop(columns=['preceding_index'], inplace=True)


In [42]:
selectedposts.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI
1175,t3_17feeu,I'm rooted on an S3 and debating what ROM I'd ...,android,What ROM are you currently running and what do...,none,question,True,NaN,RipperDaVe,[],"[0.3085225224494934, 0.0025453902781009674, -0..."
1176,t1_c84zkny,ParanoidKANGdroid. Has AOKP paranoid android a...,android,What ROM are you currently running and what do...,t3_17feeu,answer,None,1.0,None,"[""AOKP paranoid android and CM10 can be combin...","[0.6152559518814087, -0.22806480526924133, -0...."
1177,t1_c84znti,Isn't this kind of like PAC (Paranoid/AOKP/CM)?,android,What ROM are you currently running and what do...,t1_c84zkny,question,None,2.0,OmegaVesko,"[""This is similar to PAC (Paranoid/AOKP/CM).""]","[0.4787346422672272, -0.1805042177438736, -0.4..."
1178,t1_c850f55,"Omg. That's what it should be called, way cool...",android,What ROM are you currently running and what do...,t1_c84znti,appreciation,None,3.0,None,"[""ParanoidKANGdroid is not as cool as the sugg...","[0.35291609168052673, -0.2874097526073456, -0...."
1179,t1_c850if1,http://forum.xda-developers.com/showthread.php...,android,What ROM are you currently running and what do...,t1_c850f55,elaboration,None,4.0,OmegaVesko,[],"[0.3085225224494934, 0.0025453902781009674, -0..."


In [47]:
GROUPER='thread_title'
groupeddata = selectedposts.groupby(GROUPER)
groupeddata['preceding_index'] = groupeddata.body.shift(1).index


TypeError: 'DataFrameGroupBy' object does not support item assignment

In [53]:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = selectedposts[:5].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            print(index)
            print(threadset)

100%|██████████| 1/1 [00:00<00:00, 942.12it/s]

1176
1175
1177
1176
1178
1177
1179
1178


In [54]:
#test low temperature claims per comment:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = selectedposts[:5].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            try: 
                chunked_result.append(
                    pd.DataFrame(
                        data=[
                            requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODEL,
                                    'system': SYSTEM_expansion,
                                    'prompt': f'"Thread":\n<{df["body"][:threadset].to_list()}>, "Target reply":<<{df["body"][threadset:index].to_list()}>>',
                                    'options': options_low
                                    }).json()['response']                       
                        ],
                        columns=['post_expansion']
                    )
                )
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")
    

100%|██████████| 1/1 [00:07<00:00,  7.93s/it]


In [55]:
chunked_result

[                                      post_expansion
 0  "Looks like we have a new contender for the ul...,
                                       post_expansion
 0  "Looks like we have a new contender for the ul...,
                                       post_expansion
 0  "Looks like we have a new contender for the ul...,
                                       post_expansion
 0  "Looks like we have a new contender for the ul...]

In [56]:
expanded_posts = pd.concat(chunked_result, ignore_index=True)
print(expanded_posts)

                                      post_expansion
0  "Looks like we have a new contender for the ul...
1  "Looks like we have a new contender for the ul...
2  "Looks like we have a new contender for the ul...
3  "Looks like we have a new contender for the ul...


In [59]:
expanded_posts.post_expansion.to_list()

['"Looks like we have a new contender for the ultimate ROM combination! PAC (Paranoid/AOKP/CM) is indeed a game-changer, offering unparalleled customization and stability on Jellybean. If you\'re looking to take your S3 experience to the next level, give it a try!"',
 '"Looks like we have a new contender for the ultimate ROM combination! With AOKP\'s customization and CM10\'s stability, it\'s no wonder people are raving about this genius mashup. And with Jellybean support to boot, it\'s a game-changer for S3 users (and others too!) who want the best of both worlds."',
 '"Looks like we have a new contender for the ultimate ROM combination! With AOKP\'s customization and CM10\'s stability, it\'s no wonder people are raving about this genius mashup. And with Jellybean support to boot, it\'s a game-changer for S3 users (and others too!) who want the best of both worlds."',
 '"Looks like we have a new contender for the ultimate ROM combination! With AOKP\'s customization and CM10\'s stabili